This notebook checks the behaviour of the embedding space

In [1]:
import pickle
import random
import glob
import os
import sys
import scipy.sparse
import re

import numpy as np
import pandas as pd

from annoy import AnnoyIndex
from scipy.spatial import distance

scriptpath = "../../"
sys.path.append(os.path.abspath(scriptpath))
from helpers.helpers import *

In [2]:
'''
Retrieve the array obtained by apllying the dimentionality reduction algorithm
graph_matrix: SHAPE: (channels, n_comp)

PARAMETER:
    - file_path: the path where the embedding graph is stored

RETURN: 
    - df: DataFrame representing the graph in the embedding space
'''
def get_dataframe_in_embedding_space(file_path):
    graph_matrix = np.load(file_path)
    graph_matrix = graph_matrix['arr_0']
    df = pd.DataFrame(graph_matrix)
    df = df.rename(lambda x: 'dr'+str(x), axis='columns')
    return df

In [3]:
# Selected channels and id-index mapping
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

### Find k closest channel using annoy library

First to check how good is the embedding space, we are going to choose a channel and it's k closest channels in the embedding space. By looking at these channels in the YouTube website, we should get similar channels if the embedding space is good.

In [14]:
channelcrawler = pd.read_csv("/dlabdata1/youtube_large/channelcrawler.csv")
channelcrawler['channel_id'] = channelcrawler['link'].str.split('/').str[-1]
channelcrawler = channelcrawler[channelcrawler['channel_id'].apply(lambda row: row in channels_id)]

In [15]:
channelcrawler['category'].unique()

array(['Comedy', 'Gaming', 'Film and Animation', 'Science & Technology',
       'Music', 'Education', 'Sports', 'People & Blogs', 'Entertainment',
       'Nonprofits & Activism', 'News & Politics', 'Travel & Events',
       'Howto & Style', 'Autos & Vehicles', 'Pets & Animals', nan],
      dtype=object)

In [16]:
'''
Retrieve the array obtained by apllying the dimentinality reductin algorithm
graph_matrix: SHAPE: (channels, n_comp)

PARAMETERS:
    - df_embedding: DataFrame representing the graph in the embedding space
    - n_comp: number of components to use after the dimentionalit reduction

RETURN: The annoy index
'''
def get_annoy_index(df):
    index = AnnoyIndex(df.shape[1], "euclidean")  # Length of item vector that will be indexed
    df.apply(lambda row: index.add_item(row.name, np.array(row)), axis = 1)
    index.build(100) # 100 trees
    return index

In [17]:
def get_k_nearest_neighbors(path, ref_index_channel, k = 20):
    df = get_dataframe_in_embedding_space(path)
    index = get_annoy_index(df)
    nearest_neighbors_index = index.get_nns_by_item(ref_index_channel, k)
    nearest_neighbors_id = [dict_ind_channel[val] for val in nearest_neighbors_index]
    return nearest_neighbors_id

In [18]:
channelcrawler[channelcrawler['name'] == 'TEDx Talks']

,category,join_date,link,name,subscribers,videos,channel_id
12422,Nonprofits & Activism,2009-06-23,http://www.youtube.com/channel/UCsT0YIqwnpJCM-...,TEDx Talks,20700000,141627,UCsT0YIqwnpJCM-mx7-gSA4Q


In [14]:
path = '/dlabdata1/youtube_large/jouven/channel_embedding/channels_by_channels_normalized_comments_more_10k/reduced_fpca_200.npz'
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCMpOz2KEfkSdd5JeIJh_fxw'], k= 40)


In [15]:
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCMpOz2KEfkSdd5JeIJh_fxw,Howto & Style,2009-01-18,http://www.youtube.com/channel/UCMpOz2KEfkSdd5...,Shaaanxo,3215340,1422
1,UCLFW3EKD2My9swWH4eTLaYw,Howto & Style,2012-03-06,http://www.youtube.com/channel/UCLFW3EKD2My9sw...,Chloe Morello,2700000,518
2,UCXbQzhqSvgVZTUyi1T4AU3w,Howto & Style,2011-08-31,http://www.youtube.com/channel/UCXbQzhqSvgVZTU...,Lauren Curtis,3530000,379
3,UCK-4JyAcQYBasFAe367on7w,Howto & Style,2009-05-29,http://www.youtube.com/channel/UCK-4JyAcQYBasF...,RachhLoves,1450000,851
4,UCUt0ZA6l_EidUnBFMR9BZig,Howto & Style,2009-11-28,http://www.youtube.com/channel/UCUt0ZA6l_EidUn...,SMLx0,1000000,645
5,UCOtoxvHLKrIlWbt4MRBWfbQ,Howto & Style,2013-02-28,http://www.youtube.com/channel/UCOtoxvHLKrIlWb...,RawBeautyKristi,751000,627
6,UCQWy33JxT07WWPdGBIuToPw,Howto & Style,2010-11-30,http://www.youtube.com/channel/UCQWy33JxT07WWP...,LoveMelisaMichelle,571000,618
7,UCJttQwY6KBhwov_2wKywtqA,Howto & Style,2010-06-27,http://www.youtube.com/channel/UCJttQwY6KBhwov...,NikkiPhillippi,1360000,959
8,UCFghPtzFcmyDiID1ASMA4Dg,Howto & Style,2008-07-03,http://www.youtube.com/channel/UCFghPtzFcmyDiI...,Lisa Eldridge,1930000,320
9,UCPG6A5tNaPfv2SRNW2beq5Q,Entertainment,2011-11-13,http://www.youtube.com/channel/UCPG6A5tNaPfv2S...,Shani Grimmond,1590000,423


In [9]:
path = '/dlabdata1/youtube_large/jouven/channel_embedding/channels_by_channels_normalized_comments_more_10k/reduced_fpca_200.npz'
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCZyCposXwcyopaACep44maQ'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCZyCposXwcyopaACep44maQ,Howto & Style,2011-09-29,http://www.youtube.com/channel/UCZyCposXwcyopa...,Alex Costa,2120000,543
1,UCZIIRX8rkNjVpP-oLMHpeDw,Sports,2012-03-02,http://www.youtube.com/channel/UCZIIRX8rkNjVpP...,Calisthenicmovement,2152446,208
2,UCaBqRxHEMomgFU-AkSfodCw,People & Blogs,2013-08-31,http://www.youtube.com/channel/UCaBqRxHEMomgFU...,CHRIS HERIA,1230000,102
3,UCbq8_4_mFAx_rzDF5VT7MJw,Howto & Style,2013-11-11,http://www.youtube.com/channel/UCbq8_4_mFAx_rz...,BluMaan,1530000,381
4,UCZzhmrvDP3ueh7bYaTL4b7w,Sports,2016-09-24,http://www.youtube.com/channel/UCZzhmrvDP3ueh7...,AlphaShred TV,418000,326
5,UCZGNLDywn8hgzqrC9Mlz_Pw,Entertainment,2013-08-27,http://www.youtube.com/channel/UCZGNLDywn8hgzq...,Tai Lopez,1300000,138
6,UCYlMlavtN2ppRd7a6U_4ymg,People & Blogs,2017-02-07,http://www.youtube.com/channel/UCYlMlavtN2ppRd...,Stronger Than The Wo...,357000,307
7,UC_Ih0f1H_eyHeLLT0Tzuh8g,Sports,2006-05-22,http://www.youtube.com/channel/UC_Ih0f1H_eyHeL...,Simeon Panda,1220000,240
8,UC_9PPhlRLYdJby2OjNYfBEA,Entertainment,2014-06-20,http://www.youtube.com/channel/UC_9PPhlRLYdJby...,RiskyRobTV,333000,107
9,UCaaApS4TMI3KueyAkIQv1XQ,Howto & Style,2011-10-03,http://www.youtube.com/channel/UCaaApS4TMI3Kue...,AWxInc,412000,373


In [10]:
path = '/dlabdata1/youtube_large/jouven/channel_embedding/channels_by_channels_normalized_comments_more_10k/reduced_fpca_200.npz'
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCAql2DyGU2un1Ei2nMYsqOA'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCAql2DyGU2un1Ei2nMYsqOA,News & Politics,2015-03-17,http://www.youtube.com/channel/UCAql2DyGU2un1E...,Donald J Trump,189000,222
1,UCBTNyrZoiTweJ1PZsJgdWTA,News & Politics,2012-02-07,http://www.youtube.com/channel/UCBTNyrZoiTweJ1...,Tulsi Gabbard,94600,464
2,UC3o7kbpTUQ5-0WTMIp8sVwA,News & Politics,2006-01-23,http://www.youtube.com/channel/UC3o7kbpTUQ5-0W...,GOP,68700,969
3,UCBPBm0lmc-W0m6X-dXYTjqg,News & Politics,2007-11-12,http://www.youtube.com/channel/UCBPBm0lmc-W0m6...,news672,25571,4068
4,UCAHlQMx1f3TbRImhHHGPNKA,News & Politics,2010-09-15,http://www.youtube.com/channel/UCAHlQMx1f3TbRI...,Freedom Watch,45100,629
5,UC8aIq_l3wnoWDVu3sasgoew,News & Politics,2015-09-16,http://www.youtube.com/channel/UC8aIq_l3wnoWDV...,Let The Madness Begi...,15700,630
6,UCBLGQrOOfs5l7fay66B2-3Q,People & Blogs,2017-03-29,http://www.youtube.com/channel/UCBLGQrOOfs5l7f...,Silver Report Uncut,41200,761
7,UCAR0Oi4L0Om4F26uwpANgCg,Science & Technology,2012-01-21,http://www.youtube.com/channel/UCAR0Oi4L0Om4F2...,1000frolly PhD,31300,167
8,UCAVu4nbyK-IET2eFc9qqpAQ,Entertainment,2007-02-10,http://www.youtube.com/channel/UCAVu4nbyK-IET2...,Jason Asselin,36203,2006
9,UC6ZFN9Tx6xh-skXCuRHCDpQ,News & Politics,2009-11-30,http://www.youtube.com/channel/UC6ZFN9Tx6xh-sk...,PBS NewsHour,1380000,26035


In [12]:
dict_channel_ind['UC9pXxdNqCc2zjgRXSoowNNg']

9629

In [11]:
path = '/dlabdata1/youtube_large/jouven/channel_embedding/channels_by_channels_normalized_comments_more_10k/reduced_fpca_200.npz'
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UC9pXxdNqCc2zjgRXSoowNNg'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UC9pXxdNqCc2zjgRXSoowNNg,News & Politics,2014-07-09,http://www.youtube.com/channel/UC9pXxdNqCc2zjg...,A Blue Dot In Texas,10600,646
1,UC6xM7GvKd85duq5fw8qKBOQ,News & Politics,2006-01-22,http://www.youtube.com/channel/UC6xM7GvKd85duq...,Matt Orfalea,20800,232
2,UCB80iBRAaz8sYccjiZUobcw,People & Blogs,2007-12-12,http://www.youtube.com/channel/UCB80iBRAaz8sYc...,Storm,10000,312
3,UCuooZI9u0ZSGB58LMrT_T0w,People & Blogs,2006-06-07,http://www.youtube.com/channel/UCuooZI9u0ZSGB5...,Njenje Media TV,56400,672
4,UCuJ_tm9InltvVwcf71sn9zw,Travel & Events,2007-04-19,http://www.youtube.com/channel/UCuJ_tm9InltvVw...,Travelling with Bruc...,33800,1085
5,UCF_mQgKpqDBPDWF-pHRTMgg,News & Politics,2011-06-14,http://www.youtube.com/channel/UCF_mQgKpqDBPDW...,David Von Pein's JFK...,25500,899
6,UC2oIvS_wzweWauJea77q1Dg,Travel & Events,2015-05-22,http://www.youtube.com/channel/UC2oIvS_wzweWau...,The Motorhome Experi...,27500,431
7,UCuKotzG8QrGmy-lSExEN9Pw,Entertainment,2011-06-22,http://www.youtube.com/channel/UCuKotzG8QrGmy-...,DMRnews,27100,1318
8,UCtvGQztIvHC3jt-JY7a7bIQ,Music,2010-07-27,http://www.youtube.com/channel/UCtvGQztIvHC3jt...,gullivior,21700,5436
9,UC7UAVEyiMRCRb8dWCpxC5qA,Travel & Events,2014-01-04,http://www.youtube.com/channel/UC7UAVEyiMRCRb8...,Casey Roman,25800,164


In [9]:
path = '/dlabdata1/youtube_large/jouven/channel_embedding/channels_by_channels_normalized_comments_more_10k/reduced_fpca_200.npz'
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCAuUUnT6oDeKwE6v1NGQxug'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCAuUUnT6oDeKwE6v1NGQxug,People & Blogs,2006-12-06,http://www.youtube.com/channel/UCAuUUnT6oDeKwE...,TED,14800000,3105
1,UC7IcJI8PUf5Z3zKxnZvTBog,Education,2010-05-18,http://www.youtube.com/channel/UC7IcJI8PUf5Z3z...,The School of Life,4870000,724
2,UC9RM-iSvTu1uPJb8X5yp3EQ,Education,2010-02-11,http://www.youtube.com/channel/UC9RM-iSvTu1uPJ...,Wendover Productions...,2220000,104
3,UC7_gcs09iThXybpVgjHZ_7g,Education,2015-02-09,http://www.youtube.com/channel/UC7_gcs09iThXyb...,PBS Space Time,1770000,211
4,UCBa659QWEk1AI4Tg--mrJ2A,Education,2006-05-17,http://www.youtube.com/channel/UCBa659QWEk1AI4...,Tom Scott,1890000,538
5,UCB6PV0cvJpzlcXRG7nz6PpQ,Science & Technology,2009-11-09,http://www.youtube.com/channel/UCB6PV0cvJpzlcX...,Motherboard,1400000,298
6,UC9pgQfOXRsp4UKrI8q0zjXQ,Education,2007-02-23,http://www.youtube.com/channel/UC9pgQfOXRsp4UK...,Lindybeige,907000,625
7,UCAL3JXZSzSm8AlZyD3nQdBA,Science & Technology,2015-05-01,http://www.youtube.com/channel/UCAL3JXZSzSm8Al...,Primitive Technology...,9797999,48
8,UC9uD-W5zQHQuAVT2GdcLCvg,Science & Technology,2015-01-13,http://www.youtube.com/channel/UC9uD-W5zQHQuAV...,Science Insider,963425,364
9,UCA071Pllf2wk-B8Rkwt47bQ,Science & Technology,2012-01-24,http://www.youtube.com/channel/UCA071Pllf2wk-B...,Anonymous Official,2580000,409


In [19]:
path = '/dlabdata1/youtube_large/jouven/channel_embedding/channels_by_channels_normalized_comments_more_10k/reduced_fpca_200.npz'
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCwTkM6CvIsYFaFiMKIKCqHw'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCwTkM6CvIsYFaFiMKIKCqHw,Film and Animation,2005-11-27,http://www.youtube.com/channel/UCwTkM6CvIsYFaF...,James Bond 007,115000,407
1,UCwQJJVNbn_Tqze89xf83nhw,Entertainment,2007-09-12,http://www.youtube.com/channel/UCwQJJVNbn_Tqze...,The Asylum - Officia...,11200,210
2,UCwSIJCMWZC5GDM59wj7pMsg,Entertainment,2006-05-21,http://www.youtube.com/channel/UCwSIJCMWZC5GDM...,Prime Video UK,53200,996
3,UCwOj_g5BJXjrcqKszOZAAYA,Entertainment,2015-08-10,http://www.youtube.com/channel/UCwOj_g5BJXjrcq...,Late to the Party,45300,975
4,UCxxm-M-ca9WJ2k4tq8eP7og,Gaming,2011-03-08,http://www.youtube.com/channel/UCxxm-M-ca9WJ2k...,Johnny Jawbone,50500,143
5,UCx8Hvu3edWIwQhQZVvMT9aQ,Entertainment,2008-05-22,http://www.youtube.com/channel/UCx8Hvu3edWIwQh...,medici.tv,197434,3005
6,UCwBhXADStizYpJyXDUZkATg,Music,2008-04-02,http://www.youtube.com/channel/UCwBhXADStizYpJ...,Kellee Maize,20000,48
7,UCxc5H6i4wBX5D6sGbxEuBDg,Film and Animation,2016-11-23,http://www.youtube.com/channel/UCxc5H6i4wBX5D6...,StoryDive,44200,69
8,UCwLuX0LoZmlIFpfKvCHyELw,Entertainment,2014-09-15,http://www.youtube.com/channel/UCwLuX0LoZmlIFp...,30 Rock Official,31100,240
9,UCxzzanTaBBNwZqPdLmxFr5w,Comedy,2014-09-12,http://www.youtube.com/channel/UCxzzanTaBBNwZq...,Best of Simpsons Cha...,53400,68


In [20]:
path = '/dlabdata1/youtube_large/jouven/channel_embedding/channels_by_channels_normalized_comments_more_10k/reduced_fpca_200.npz'
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCISF5OGuAtSLNF24TKTnXag'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCISF5OGuAtSLNF24TKTnXag,Film and Animation,2013-12-17,http://www.youtube.com/channel/UCISF5OGuAtSLNF...,Magpiepony,736000,304
1,UCHKDtSVVEc686EW7jRSCpZA,Entertainment,2007-04-01,http://www.youtube.com/channel/UCHKDtSVVEc686E...,Scribbler Production...,327000,1767
2,UCH-blHqGEo1dE35f6_y0j5A,Music,2015-10-31,http://www.youtube.com/channel/UCH-blHqGEo1dE3...,Maloney,188882,159
3,UCLFd05hT4i87cu-IjSIwLUA,Gaming,2014-12-20,http://www.youtube.com/channel/UCLFd05hT4i87cu...,Starbeam,363000,93
4,UCJDFINJB23y0MeXO6BX7EFw,Film and Animation,2011-08-18,http://www.youtube.com/channel/UCJDFINJB23y0Me...,Rossali,497000,149
5,UCMBpJ5kRKU1X5QBc_hmFyBg,Film and Animation,2013-03-14,http://www.youtube.com/channel/UCMBpJ5kRKU1X5Q...,YoshTea,131000,155
6,UC8Wj98MR_oUHBpTjLsE3HuA,Film and Animation,2010-10-10,http://www.youtube.com/channel/UC8Wj98MR_oUHBp...,Pinkie Rose,247000,167
7,UCUmsZrGGKvQD8E3kSXQNs2g,Film and Animation,2011-08-17,http://www.youtube.com/channel/UCUmsZrGGKvQD8E...,QuestionedTurkey,158000,751
8,UCKZphsoYAAIe2o2uItTQbOw,Film and Animation,2016-10-25,http://www.youtube.com/channel/UCKZphsoYAAIe2o...,Shavs Media Producti...,143000,202
9,UCMOjuRCO7hLk4DNYXX8Tz4A,Film and Animation,2011-10-16,http://www.youtube.com/channel/UCMOjuRCO7hLk4D...,MarlineAnimates,225000,16


### Channels selected over the whole comments dataset
We randomly choose 10 000 users over the dataset.
For each user, we then pick two channels at random in the set of channels this user commented in.

In [21]:
with open("/dlabdata1/youtube_large/jouven/channels_more_10k/channels_tuple_user_walk.pkl",'rb') as f:
     channels_tuple = pickle.load(f)
f.close()

#### Random walk

In [22]:
def get_random_walk(df_embedding):
    with open("/dlabdata1/youtube_large/jouven/channels_more_10k/channels_tuple_random_walk.pkl",'rb') as f:
         random_walk_channels = pickle.load(f)
    f.close()
    random_walk_distance = 0
    for val in random_walk_channels:
        random_walk_distance += distance.euclidean(df_embedding.iloc[val[0]], df_embedding.iloc[val[1]])
    return random_walk_distance

#### Compute metrics: users walk distance and relative nearest neighbor ranking 

In this section we want to measure the euclidian distance of a user walk compared to a random walk.

user walk: Euclidean distance in the embedding space between two randomly channels of a user.
random walk: Euclidean distance in the embedding space between two randomly channels.
position: Position of a channel taken from user u relatively of another channel taken from the same user in terms of its nearest neighbor ranking.


In [23]:
'''
Get the position of ref_channel relative to second_channel in terms of its nearest neighbors ranking.
PARAMETER:
    - ref_channel: The reference channel on which wwe compute it's k nearest neighbor
    - second_channel: The channel where we compute it's ranking relatively to ref_channel
    - dist: Euclidean distance between ref_channel and second_channel
    - index: annoy index
    - df_embedding: DataFrame representing the embedding space

RETURN: The position of second_channel relatively to ref_channel in terms of it's ranking

'''
def get_ranking_position_between_channels(ref_channel, second_channel, dist, index, df_embedding):
    
    nearest_neighbors_index = index.get_nns_by_item(ref_channel, len(df_embedding), search_k = 100000000)
    dist_k_th_nearest = distance.euclidean(df_embedding.iloc[ref_channel], 
                                           df_embedding.iloc[nearest_neighbors_index[len(nearest_neighbors_index)-1]])
    for i in range(0, len(nearest_neighbors_index)):
        if nearest_neighbors_index[i] == second_channel:
            return i
    

In [24]:
users_walk_tab = []
ranking_position_tab = []

len_random_set = len(channels_tuple)
len_embedding = len(channels_id)

#path = '/dlabdata1/youtube_large/jouven/channel_embedding/limited_normalized_50/'
#files = glob.glob(path + '*.npz') 
files = ['/dlabdata1/youtube_large/jouven/channel_embedding/channels_by_channels_normalized_comments_more_10k/reduced_fpca_200.npz']
for file in files: 
    print('file ', file)
    df_embedding = get_dataframe_in_embedding_space(file)
    n_comp = df_embedding.shape[1]
    print('n_comp ', n_comp)
    random_walk_distance = get_random_walk(df_embedding)
        
    index = get_annoy_index(df_embedding)

    users_walk = 0
    ranking_position = 0

    for ref_channel, second_channel in channels_tuple:
        dist = distance.euclidean(df_embedding.iloc[ref_channel], df_embedding.iloc[second_channel])
        users_walk += dist
        ranking_position += get_ranking_position_between_channels(ref_channel, second_channel, dist, index, df_embedding)
    
    users_walk_tab.append(users_walk/random_walk_distance)
    ranking_position_tab.append(ranking_position/(len_random_set*len_embedding))


file  /dlabdata1/youtube_large/jouven/channel_embedding/channels_by_channels_normalized_comments_more_10k/reduced_fpca_200.npz
n_comp  200


In [25]:
users_walk_tab

[8.968204126584432]

In [26]:
ranking_position_tab

[0.7003344443664772]